Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas 
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [1]:
import pandas as pd
wage = pd.read_csv('wage.csv')

3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [2]:
# меняем тип данных столбца gender на строки
wage['gender'] = wage['gender'].astype(str)

wage.loc[wage['gender'] == '1', 'gender'] = 'M'
wage.loc[wage['gender'] == '0', 'gender'] = 'F'

4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [3]:
wage.groupby('gender')['wage'].mean()

gender
F    40855.747261
M    46815.944005
Name: wage, dtype: float64

5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [4]:
# считаем сколько раз встречается каждый person_id
# сохраняем их в отдельный Index
pers_cnt = wage['person_id'].value_counts()
duplic_id = pers_cnt[pers_cnt > 1].index
duplic_id

Index([12, 13, 14, 15, 16, 17], dtype='int64', name='person_id')

In [5]:
# выбираем из таблицы wage нужные записи
# считаем количество уникальных значений wage
# для каждого person_id
duplicates = wage[wage['person_id'].isin(duplic_id)]
duplicates.groupby('person_id')['wage'].nunique()

person_id
12    1
13    1
14    1
15    1
16    1
17    1
Name: wage, dtype: int64

Видим, что людей у которых указано две разные зарплаты нет, поэтому просто удаляем все дубликаты

In [6]:
# удаляем дубликаты и проверяем, что в таблице
# больше нет повторяющихся людей
wage.drop_duplicates(inplace=True)
pers_cnt = wage['person_id'].value_counts()
print(pers_cnt[pers_cnt > 1])


Series([], Name: count, dtype: int64)


6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [7]:
wage['wage'].describe()

count      1000.000000
mean      43694.227404
std       55352.539343
min     -287418.645743
25%       14489.682367
50%       27309.529498
75%       52021.080258
max      755320.874132
Name: wage, dtype: float64

In [8]:
# выбираем записи с отрицательной зарплатой
# и удаляем эти записи из таблицы
# проверяем, что они действительно удалились
strange = wage[wage['wage'] < 0]
wage.drop(strange.index, inplace=True)
wage[wage['wage'] < 0]

,person_id,gender,wage


7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [9]:
bonus = pd.read_csv('bonus.csv', sep=';')
bonus

,person_id,bonus
0,905,85059.638382
1,836,7703.346074
2,287,3120.269742
3,548,5347.987142
4,575,137257.490614
...,...,...
445,488,28102.252903
446,913,55549.347647
447,616,620397.407705
448,110,14086.067488


8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [16]:
df = wage.merge(bonus, how='outer', on='person_id')
df

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,NaN
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,NaN
...,...,...,...,...
994,995,M,66503.737185,3.452137e+03
995,996,M,9972.956272,3.892790e+05
996,997,F,104504.616392,5.380978e+04
997,998,M,98927.903076,NaN


9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

In [19]:
# записи, где gender = 0 возникли при слияние таблиц
# это люди, у которых была отрицательная зарплата
df = df.fillna(0)
df['total'] = df['wage'] * 12 + df['bonus']
df.groupby('gender')['total'].agg(['median', 'mean'])

,median,mean
gender,,
0,142936.832502,142936.832502
F,347622.913892,570746.139432
M,437499.824868,657142.490282


10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [18]:
df.to_csv('final.csv', index=False)